In [61]:
from bs4 import BeautifulSoup
import requests
import json

def parse_html_table(url):
            req = requests.get(url)
            soup = BeautifulSoup(req.text, 'lxml')
            table = soup.find_all('table')[0] # Grab the first table
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            # columns = column_names if len(column_names) > 0 else range(0,n_columns) => operating
            if len(column_names) > 0:
                columns = column_names
            else:
                columns = range(0,n_columns)
                
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    value = str(column.get_text())
                    df.iat[row_marker,column_marker] = value.replace('\n','')
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df   




In [63]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
postal = parse_html_table(url)
postal.head()

2020-03-07 20:06:08 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): en.wikipedia.org:443
2020-03-07 20:06:09 [urllib3.connectionpool] DEBUG: https://en.wikipedia.org:443 "GET /wiki/List_of_postal_codes_of_Canada:_M HTTP/1.1" 200 14956


,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
